In [1]:
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tokenizers import Tokenizer

In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Sethi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sethi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sethi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
og_data = pd.read_csv("data\output_chunk_7.csv")

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
C:\Users\Sethi\AppData\Local\Temp\ipykernel_31180\861610438.py:1: SyntaxWarning: invalid escape sequence '\o'
  og_data = pd.read_csv("data\output_chunk_7.csv")


In [4]:
og_data_98 = og_data.sample(frac = 0.98) # work data for assignment
og_data_02 = og_data.drop(og_data_98.index) # test data to be used by prof

In [5]:


def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
    # call stop words and remove them
    stop_words = stopwords.words('english') 
    removed_stopwords_text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    # Perform stemming
    stemmer = nltk.SnowballStemmer("english")
    return ' '.join(stemmer.stem(word) for word in removed_stopwords_text.split(' '))


og_data_test = og_data_98
og_data_test["text"] = og_data_98["text"].apply(clean_text)
og_data_test.head()



,category,text
6721,unrest,least peopl kill burkina faso deadliest attac...
2033,environmental,endang hawaiian monk seal recent transport pap...
2925,humanInterest,franc learment saugeen district senior school...
1196,disaster,uae ministri health prevent friday report cas...
2425,health,gasp air patient haiti infect coronavirus die ...


In [6]:
og_data_98.head()

,category,text
6721,unrest,least peopl kill burkina faso deadliest attac...
2033,environmental,endang hawaiian monk seal recent transport pap...
2925,humanInterest,franc learment saugeen district senior school...
1196,disaster,uae ministri health prevent friday report cas...
2425,health,gasp air patient haiti infect coronavirus die ...


In [7]:
og_data_test.head()

,category,text
6721,unrest,least peopl kill burkina faso deadliest attac...
2033,environmental,endang hawaiian monk seal recent transport pap...
2925,humanInterest,franc learment saugeen district senior school...
1196,disaster,uae ministri health prevent friday report cas...
2425,health,gasp air patient haiti infect coronavirus die ...


In [8]:
from sklearn.model_selection import train_test_split

X= og_data_test["text"]
y= og_data_test["category"]

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=.2,random_state=42)

# Multinomial Naive Bayes with Bag of Words
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score,classification_report

model = make_pipeline(CountVectorizer(), MultinomialNB())

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"MultinomialNB with Bag of Words accuracy: {accuracy:.3f}")

MultinomialNB with Bag of Words accuracy: 0.672


In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

cv_scores = cross_val_score(model, X, y, cv=StratifiedKFold(n_splits=3, shuffle=True), scoring='accuracy')

print(f"Cross-Validation Scores:{cv_scores}")

print(f"Mean Accuracy: {np.mean(cv_scores):.2f}")

Cross-Validation Scores:[0.6704932  0.65561224 0.66369048]
Mean Accuracy: 0.66


In [10]:
import tensorflow as tf
import transformers

In [11]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [12]:
#tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased')

In [13]:
#Start
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

# Encode labels
le = LabelEncoder()
y = le.fit_transform(og_data_98['category'])

# Create pipeline
model2 = make_pipeline(
    TfidfVectorizer(max_features=5000),
    LogisticRegression(max_iter=1000)
)

# Train
model2.fit(og_data_98['text'], y)

# Evaluate
print("Model 2 Accuracy:", model2.score(og_data_98['text'], y))




Model 2 Accuracy: 0.9030612244897959


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

og_data_98['cleaned_text'] = og_data_98['text'].apply(preprocess_text)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(og_data_98['cleaned_text'])
tokenizer.num_words = 5000
sequences = tokenizer.texts_to_sequences(og_data_98['cleaned_text'])
X = pad_sequences(sequences, maxlen=200)


# Build model
model3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

model3.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train
history = model3.fit(
    X, y,
    batch_size=32,
    epochs=20,
    validation_split=0.2
)

# Evaluate
print("Model 3 Validation Accuracy:", max(history.history['val_accuracy']))

Epoch 1/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.0929 - loss: 2.8448 - val_accuracy: 0.2018 - val_loss: 2.5350
Epoch 2/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.2433 - loss: 2.3137 - val_accuracy: 0.3831 - val_loss: 1.9933
Epoch 3/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.4712 - loss: 1.6856 - val_accuracy: 0.4788 - val_loss: 1.7485
Epoch 4/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.6264 - loss: 1.2242 - val_accuracy: 0.4653 - val_loss: 1.8403
Epoch 5/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.6892 - loss: 1.0148 - val_accuracy: 0.5297 - val_loss: 1.7411
Epoch 6/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.7860 - loss: 0.7125 - val_accuracy: 0.5333 - val_loss: 1.9066
Epoch 7/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.8607 - loss: 0.4779 - val_accuracy: 0.5375 - val_loss: 2.0017
Epoch 8/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.9157 - loss: 0.3120 - val_accu

In [24]:
# First create and save all models
import pandas as pd
import re
import string
import joblib
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load and preprocess data
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

og_data = pd.read_csv("data/output_chunk_7.csv")
og_data['cleaned_text'] = og_data['text'].apply(preprocess_text)

# Split data
og_data_98 = og_data.sample(frac=0.98)
og_data_02 = og_data.drop(og_data_98.index)

# Encode labels
le = LabelEncoder()
y = le.fit_transform(og_data_98['category'])

# Model 1: Original Model (Naive Bayes)
model1 = make_pipeline(
    TfidfVectorizer(max_features=5000),
    MultinomialNB()
)
model1.fit(og_data_98['cleaned_text'], y)
joblib.dump(model1, 'NaiveBayes.pkl')

# Model 2: Logistic Regression
model2 = make_pipeline(
    TfidfVectorizer(max_features=5000),
    LogisticRegression(max_iter=1000)
)
model2.fit(og_data_98['cleaned_text'], y)
joblib.dump(model2, 'logreg_model.pkl')

# Model 3: LSTM
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(og_data_98['cleaned_text'])
sequences = tokenizer.texts_to_sequences(og_data_98['cleaned_text'])
X_lstm = pad_sequences(sequences, maxlen=200)

model3 = Sequential([
    Embedding(5000, 128),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])
model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.fit(X_lstm, y, epochs=10, validation_split=0.2)
model3.save('lstm_model.h5')

# Save preprocessing artifacts
joblib.dump(le, 'label_encoder.pkl')
with open('tokenizer.json', 'w') as f:
    f.write(tokenizer.to_json())

# Loading function for all models
def load_all_models():
    model1 = joblib.load('original_model.pkl')
    model2 = joblib.load('logreg_model.pkl')
    model3 = load_model('lstm_model.h5')
    le = joblib.load('label_encoder.pkl')
    
    with open('tokenizer.json', 'r') as f:
        tokenizer = tokenizer_from_json(f.read())
    
    return model1, model2, model3, le, tokenizer

# Prediction function for all models
def predict_all_models(text, model1, model2, model3, le, tokenizer):
    cleaned = preprocess_text(text)
    
    # Model 1 (Naive Bayes)
    pred1 = model1.predict([cleaned])[0]
    
    # Model 2 (Logistic Regression)
    pred2 = model2.predict([cleaned])[0]
    
    # Model 3 (LSTM)
    sequence = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(sequence, maxlen=200)
    pred3 = model3.predict(padded).argmax(axis=1)[0]
    
    return {
        'NaiveBayes': le.inverse_transform([pred1])[0],
        'Logistic Regression': le.inverse_transform([pred2])[0],
        'LSTM': le.inverse_transform([pred3])[0]
    }


Epoch 1/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.0733 - loss: 2.8645 - val_accuracy: 0.1629 - val_loss: 2.6396
Epoch 2/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.2012 - loss: 2.4685 - val_accuracy: 0.2493 - val_loss: 2.3583
Epoch 3/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.3177 - loss: 2.0920 - val_accuracy: 0.2854 - val_loss: 2.2513
Epoch 4/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - accuracy: 0.4614 - loss: 1.6980 - val_accuracy: 0.3414 - val_loss: 2.1650
Epoch 5/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.6103 - loss: 1.2459 - val_accuracy: 0.3683 - val_loss: 2.1650
Epoch 6/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.7282 - loss: 0.8957 - val_accuracy: 0.4108 - val_loss: 2.2827
Epoch 7/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.8413 - loss: 0.5470 - val_accuracy: 0.4207 - val_loss: 2.4244
Epoch 8/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.9119 - loss: 0.3365 - val_acc

In [25]:
# Load models once at startup
model1, model2, model3, le, tokenizer = load_all_models()

# Make predictions
sample_text = "Archaeologists discover ancient temple ruins under city center"
predictions = predict_all_models(sample_text, model1, model2, model3, le, tokenizer)

print("Predictions:", predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Predictions: {'NaiveBayes': 'arts', 'Logistic Regression': 'arts', 'LSTM': 'science'}
